# Imports and Version Control

In [18]:
import tensorflow as tf
# tf.logging.set_verbosity(tf.logging.ERROR)

import numpy as np
!pip install gym
!pip install keras==2.2.4
!pip install keras-rl
!pip install h5py
!pip install Pillow

import gym

In [19]:
import numpy as np
!pip install gymnasium
# !pip install gym[atari,accept-rom-license]
# import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

  Obtaining dependency information for gymnasium from https://files.pythonhosted.org/packages/a8/4d/3cbfd81ed84db450dbe73a89afcd8bc405273918415649ac6683356afe92/gymnasium-0.29.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/953.9 kB ? eta -:--:--
   -- ------------------------------------- 61.4/953.9 kB 1.7 MB/s eta 0:00:01
   ---------- ----------------------------- 256.0/953.9 kB 3.2 MB/s eta 0:00:01
   ------------------------------------- -- 901.1/953.9 kB 6.3 MB/s eta 0:00:01
   ---------------------------------------- 953.9/953.9 kB 6.0 MB/s eta 0:00:00


In [21]:
!pip install gymnasium[atari]

In [25]:
import gymnasium

# Train Model

In [26]:
from gym import envs
env_ids = [spec.id for spec in envs.registry.all()]
env_ids

AttributeError: 'dict' object has no attribute 'all'

In [27]:
import gym

# Initialize environment
env = gym.make('ALE/Breakout-v5')

# Number of episodes
for i_episode in range(5):
    observation = env.reset()
    for t in range(1000):
        env.render()
        action = env.action_space.sample()  # Take a random action
        observation, reward, done, info = env.step(action)
        if done:
            print(f"Episode finished after {t+1} timesteps")
            break

env.close()

FileNotFoundError: Could not find module 'c:\Users\clayb\Anaconda3\envs\deep_py\lib\site-packages\atari_py\ale_interface\ale_c.dll' (or one of its dependencies). Try using the full path with constructor syntax.

In [ ]:
# Initialize Gym environment
ENV_NAME = 'Breakout-v0'
env = gym.make(ENV_NAME)
np.random.seed(123)
env.seed(123)

In [ ]:
# Number of actions
nb_actions = env.action_space.n

In [ ]:
# Define the model architecture
input_shape = (84, 84)
window_length = 4

model = Sequential()
model.add(Permute((2, 3, 1), input_shape=(window_length,) + input_shape))
model.add(Convolution2D(32, (8, 8), strides=(4, 4)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (3, 3), strides=(1, 1)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))

In [ ]:
# Configure and compile the agent
memory = SequentialMemory(limit=1000000, window_length=window_length)
policy = EpsGreedyQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=50000,
               target_model_update=10000, policy=policy)
dqn.compile(Adam(lr=0.00025), metrics=['mae'])

In [ ]:
# Train the agent
dqn.fit(env, nb_steps=1750000, visualize=False, verbose=2)

In [ ]:
# Save the final policy network
dqn.save_weights('policy.h5', overwrite=True)

# Play Model

In [ ]:
model = Sequential()
model.add(Permute((2, 3, 1), input_shape=(window_length,) + input_shape))
model.add(Convolution2D(32, (8, 8), strides=(4, 4)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (3, 3), strides=(1, 1)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))

In [ ]:
# Configure and compile the agent
memory = SequentialMemory(limit=1000000, window_length=window_length)
policy = GreedyQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=50000,
               target_model_update=10000, policy=policy)
dqn.compile(Adam(lr=0.00025), metrics=['mae'])

In [ ]:
# Load the trained weights
dqn.load_weights('policy.h5')

In [ ]:
# Evaluate the agent
dqn.test(env, nb_episodes=10, visualize=True)